In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score
%matplotlib inline


## Линейная регрессия

Загрузите файл food_trucks.txt. В нём два столбца значений — количество жителей в городе и доход грузовика с уличной едой в этом городе.

In [ ]:
!wget https://www.dropbox.com/s/z5u3bmk21knfh6a/food_trucks.txt

In [ ]:
df = pd.read_csv("food_trucks.txt", header=None)
df.head()

In [ ]:
x1 = np.array(df[0])
x0 = np.ones(x1.shape)
y = np.array(df[1])
X = np.array([x0, x1]).transpose()
w = np.random.rand(1, 2).ravel()
epochs=3
for i in range(epochs): 
    y_hat = X @ w  # The current predicted value of Y
    dw = # your code here
    w = w - 1e-20*dw
    


In [ ]:
plt.scatter(df[0], X @ w)
plt.scatter(df[0], df[1])

In [ ]:
w

## Аналитическое решение

In [ ]:
w = np.linalg.inv(X.T@X)@X.T@y

In [ ]:
yhat = X @ w

In [ ]:
plt.scatter(x1, np.array(yhat))
plt.scatter(x1, y, c='red')

In [ ]:
w

 ${\displaystyle R^{2}}$ (Коэффициент детерминации) — это доля дисперсии объясняемая моделью 

In [ ]:
r2_score(np.squeeze(np.array(y)), np.squeeze(np.array(yhat)))

## Методы оптимизации

Как было показано на лекции, большинство методов машинного обучения сводятся к поиску параметров, которые минимизируют ошибку на тренировочной выборке:
$$
\min_{w} L(w; D)
$$
Здесь:
* $D$ — размеченная обучающая выборка, $\{x_i, y_i\}_{i=1}^N$
* $L$ — функция потерь
* $w$ — настраиваемые веса алгоритма

В более общем виде задачу можно записать так:
$$
\min_{x} f(x)
$$
Здесь:
* $x$ — вектор значений
* $f$ — функция, принимающая вектор в качестве аргумента и выдающая числовое значение.

На семинаре рассмотрим подробнее методы минимизации функции, которые рассматривались на лекции.

## Градиентный спуск

Для оптимизации возьмем простую функцию $f(x) = x^3 - 2x^2 + 2$

In [ ]:
f = lambda x: x ** 3 - 2*x ** 2 + 2
df = lambda x: 3 * x ** 2 - 4 * x # производная
x = np.linspace(-1, 2.5, 1000)
plt.plot(x, f(x))
plt.xlim([-1, 2.5])
plt.ylim([0, 3])
plt.show()

И определим функцию, которая будет оптимизировать функцию $f(x)$ градиентным спуском с заданным постоянным шагом (он же learning rate, темп обучения).

In [ ]:
def optimize_and_plot_steps(learning_rate, x_new=2, compute_learning_rate=None):
    x_old = 0
    # x_new — точка старта
    eps = 0.0001
    x_list, y_list = [x_new], [f(x_new)] # инициализируем список координат и значений функций при итерации
    
    # спускаемся, пока разница между координатами не достигла требуемой точности
    i = 0
    while abs(x_new - x_old) > eps: 
        x_old = x_new
        # считаем направление спуска
        direction = -df(x_old)
        # обновляем значение темпа обучения, если нам задана функция для этого
        if compute_learning_rate is not None:
            learning_rate = compute_learning_rate(i, learning_rate)
        # делаем шаг
        x_new = x_old + learning_rate * direction
        # запоминаем очередной шаг минимизации
        x_list.append(x_new)
        y_list.append(f(x_new))
        i += 1
        
    print("Found local min:", x_new)
    print("Steps number:", len(x_list))
    
    plt.figure(figsize=[10,3])
    
    plt.subplot(1,2,1)
    plt.scatter(x_list, y_list, c="r")
    plt.plot(x_list, y_list, c="r")
    plt.plot(x, f(x), c="b")
    plt.xlim([-1,2.5])
    plt.ylim([0,3])
    plt.title("Descent trajectory")

    plt.subplot(1,2,2)
    plt.scatter(x_list,y_list,c="r")
    plt.plot(x_list,y_list,c="r")
    plt.plot(x,f(x), c="b")
    plt.xlim([1.2,2.1])
    plt.ylim([0,3])
    plt.title("Descent trajectory (zoomed in)")
    plt.show()

Попробуем оптимизацию с шагом 0.1

In [ ]:
optimize_and_plot_steps(0.1, x_new=1)

Возьмем шаг побольше.

In [ ]:
optimize_and_plot_steps(0.4)

Что, если взять 0.5?

In [ ]:
optimize_and_plot_steps(0.5)

Застопорились в нуле, т.к. нашли точный локальный максимум. В нем производная равна нулю и мы никуда не можем сдвинуться. А если взять 0.49?

In [ ]:
optimize_and_plot_steps(0.49)

Что, если взять 0.51?

In [ ]:
optimize_and_plot_steps(0.51)

Мы улетели далеко влево. Это можно понять, распечатав значения x_new.

Теперь возьмём маленький шаг. Например, 0.05.

In [ ]:
optimize_and_plot_steps(0.05)

0.01?

In [ ]:
optimize_and_plot_steps(0.01)

Чем меньше шаг, тем медленнее мы идём к минимум (и можем вдобавок застрять по пути). Чем больше темп обучения, тем большие расстояния мы перепрыгиваем (и имеем гипотетическую возможность найти минимум получше). Хорошая стратегия — начинать с достаточно большого шага (чтобы хорошо попутешествовать по функции), а потом постепенно его уменьшать, чтобы стабилизировать процесс обучения в каком-то локальном минимуме.

Теперь будем изменять шаг динамически:
$lr(i + 1) = lr(i) * 0.9$.

In [ ]:
def compute_learning_rate(i, prev_lr):
    return prev_lr * 0.9

In [ ]:
optimize_and_plot_steps(0.45, compute_learning_rate=compute_learning_rate)

Если сравнивать с постоянным темпом обучения, то мы нашли минимум в 2 раза быстрее.

In [ ]:
optimize_and_plot_steps(0.45)

Это, конечно, искуственный пример, но такая же идея используются для обучения алгоритмов машинного обучения с миллионами параметров, функции потерь которых имеют очень сложную структуру и не поддаются визуализации.

## Продвинутые методы оптимизации

Попробуем реализовать продвинутые методы градиентного спуска!

## SGD

In [ ]:
#your code here

## SGD with momentum

In [ ]:
#your code here

## AdaGram

In [ ]:
#your code here

## RMSprop

In [ ]:
#your code here